# **Evaluación del Módulo 2 Sprint 2**
## **Inserción de datos**
### **Lidya Descals**

#### En este documento se llaman todos los métodos de la clase Universidades y se insertan los datos en la BBDD de MySQL.
---

In [1]:
import pandas as pd
import numpy as np
import requests
import mysql.connector
import sys

#Importamos la librería para sacar las coordenadas.
from geopy.geocoders import Nominatim

#Importamos la clase.
sys.path.append("../")
from src import clase_universidades as uni


---
### Extraccion

In [2]:
#Instanciamos la clase.
datos = uni.Universidades("universidades", "AlumnaAdalab")

In [3]:
df_argentina = datos.extraer_argentina()
df_argentina.head(1)

La solicitud ha tenido una respuesta con código 200, y nos devuelve el mensaje OK


,state-province,domains,name,country,web_pages,alpha_two_code
0,Buenos Aires,[atlantida.edu.ar],Universidad Atlantida Argentina,Argentina,[http://www.atlantida.edu.ar/],AR


In [4]:
df_canada = datos.extraer_canada()
df_canada.head(1)

La solicitud ha tenido una respuesta con código 200, y nos devuelve el mensaje OK


,state-province,domains,name,country,web_pages,alpha_two_code
0,Quebec,[cstj.qc.ca],Cégep de Saint-Jérôme,Canada,"[https://www.cstj.qc.ca, https://ccmt.cstj.qc....",CA


In [5]:
df_eeuu = datos.extraer_eeuu()
df_eeuu.head(1)

La solicitud ha tenido una respuesta con código 200, y nos devuelve el mensaje OK


,state-province,domains,name,country,web_pages,alpha_two_code
0,None,[marywood.edu],Marywood University,United States,[http://www.marywood.edu],US


---
### Limpieza

In [6]:
df_argentina = datos.limpieza(df_argentina)
df_argentina.head(1)

El dataframe tiene 0 duplicados.
Después de la limpieza, el dataframe tiene 0 duplicados.
El dataframe tiene un total de 22 nulos en la columna 'state_province'.
Después de la limpieza, el dataframe tiene un total de 0 nulos en la columna 'state_province'.


,state_province,name,country,web_pages,alpha_two_code
0,Buenos Aires,Universidad Atlantida Argentina,Argentina,http://www.atlantida.edu.ar/,AR


In [7]:
df_canada = datos.limpieza(df_canada)
df_canada.head(1)

El dataframe tiene 4 duplicados.
Después de la limpieza, el dataframe tiene 0 duplicados.
El dataframe tiene un total de 8 nulos en la columna 'state_province'.
Después de la limpieza, el dataframe tiene un total de 0 nulos en la columna 'state_province'.


,state_province,name,country,web_pages,alpha_two_code
0,Quebec,Cégep de Saint-Jérôme,Canada,https://www.cstj.qc.ca,CA


In [8]:
df_eeuu = datos.limpieza(df_eeuu)
df_eeuu.head(1)

El dataframe tiene 20 duplicados.
Después de la limpieza, el dataframe tiene 0 duplicados.
El dataframe tiene un total de 2208 nulos en la columna 'state_province'.
Después de la limpieza, el dataframe tiene un total de 0 nulos en la columna 'state_province'.


,state_province,name,country,web_pages,alpha_two_code
0,Unknown,Marywood University,United States,http://www.marywood.edu,US


---
### Extracción de coordenadas y unión de dataframes

In [9]:
df_latitud_longitud = datos.sacar_coordenadas(df_argentina, df_canada, df_eeuu)
df_latitud_longitud.head(1)

,state_province,latitude,longitude
0,Pennsylvania,40.969989,-77.727883


In [10]:
df_completo = datos.unir_dataframes(df_argentina, df_canada, df_eeuu, df_latitud_longitud)
df_completo.head(1)

,state_province,name,country,web_pages,alpha_two_code,latitude,longitude
295,Unknown,Brigham Young University Hawaii,United States,http://www.byuh.edu/,US,Unknown,Unknown


---
### Creación de la base de datos y sus tablas

In [11]:
datos.crear_bbdd_sql()

Conexión realizada con éxito.


'La base de datos universidades ha sido creada correctamente.'

In [12]:
datos.crear_tablas_sql()

"Las tablas 'paises' y 'universidades' han sido creadas correctamente."

---
### Inserción de datos

In [13]:
#Para insertar los datos, filtramos el df_completo para que la inserción de los valores "Unknown" no provoque un error y dejamos una única fila por cada provincia.

df_completo_paises = df_completo.drop(df_completo[df_completo['state_province'] == "Unknown"].index, inplace=False).drop_duplicates(["state_province"], inplace = False)

for fila, elem in df_completo_paises.iterrows():
    
    query_paises = f"""
                    INSERT INTO paises (nombre_pais, nombre_provincia, latitud, longitud)
                    VALUES ("{elem['country']}", "{elem['state_province']}", {elem['latitude']}, {elem['longitude']});
                    """
    
    datos.insertar_datos_sql(query_paises)

In [14]:
#Añadimos ahora los valores "Unknown" para cada país.

query_paises_unknown = f"""
                INSERT INTO paises (nombre_pais, nombre_provincia, latitud, longitud)
                VALUES ("Argentina", "Unknown", 0.0, 0.0),
                       ("Canada", "Unknown", 0.0, 0.0),
                       ("United States", "Unknown", 0.0, 0.0);
                """

datos.insertar_datos_sql(query_paises_unknown)

'Las inserción se ha realizado correctamente.'

In [15]:
#Eliminamos las comillas simples dentro de los nombres de las universidades para evitar errores en la inserción.

df_completo["name"] = df_completo["name"].str.replace("'", "")

for fila, elem in df_completo.iterrows():


    idestado = datos.extraer_idestado(elem["state_province"])
    
    
    query_universidades = f"""
                            INSERT INTO universidades (nombre_universidad, pagina_web, paises_idestado)
                            VALUES ('{elem["name"]}', '{elem["web_pages"]}', {idestado});
                            """
    
    datos.insertar_datos_sql(query_universidades)